In [0]:
# PySpark UDF (User Defined Function)

data = [("John Doeeeeeeeeeeeeeeeeeee", 25), ("Alice Wonderlandokkkkkkkkk", 30), ("Bob Builder", 22)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Display DataFrame with truncate=False
df.show(truncate=False)

+--------------------------+---+
|Name                      |Age|
+--------------------------+---+
|John Doeeeeeeeeeeeeeeeeeee|25 |
|Alice Wonderlandokkkkkkkkk|30 |
|Bob Builder               |22 |
+--------------------------+---+



In [0]:
df.show(truncate=True)

+--------------------+---+
|                Name|Age|
+--------------------+---+
|John Doeeeeeeeeee...| 25|
|Alice Wonderlando...| 30|
|         Bob Builder| 22|
+--------------------+---+



In [0]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]
df = spark.createDataFrame(data,columns)
df.show()

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



In [0]:
#Create a Python Function
de(str):
    restr=""
    arr = str.split(" ")
    for x in arr:
        restr=restr+ x[0].upper()+x[1:]+" "
    return restr
    

In [0]:
print(upper_conver("sidharth shinde"))

Sidharth Shinde 


In [0]:
#Convert a Python function to PySpark UDF
from pyspark.sql.functions import col,udf
from pyspark.sql.types import StringType
convertUDF = udf(lambda z:upper_conver(z),StringType())


In [0]:
# Using UDF with PySpark DataFrame select()
df.select(col("Seqno"),convertUDF(col("Name")).alias("Name")).show()
df.show()

+-----+-------------+
|Seqno|         Name|
+-----+-------------+
|    1|  John Jones |
|    2|Tracey Smith |
|    3| Amy Sanders |
+-----+-------------+

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



In [0]:
###  Registering PySpark UDF & use it on SQL
spark.udf.register("convertFunc",upper_conver,StringType())
df.createOrReplaceTempView("table1")
spark.sql("select Seqno,convertFunc(Name) as Name from table1").show()

+-----+-------------+
|Seqno|         Name|
+-----+-------------+
|    1|  John Jones |
|    2|Tracey Smith |
|    3| Amy Sanders |
+-----+-------------+



In [0]:
    """ null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select nvl(_nullsafeUDF(Name),'ok') from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+

+---------------------------+
|nvl(_nullsafeUDF(Name), ok)|
+---------------------------+
|_nullsafeUDF(Name)         |
|_nullsafeUDF(Name)         |
|_nullsafeUDF(Name)         |
|_nullsafeUDF(Name)         |
+---------------------------+



---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
File <command-1418464157777411>:18
     13 spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())
     15 spark.sql("select nvl('_nullsafeUDF(Name)','ok') from NAME_TABLE2") \
     16  .show(truncate=False)
---> 18 spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
     19       " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     20  .show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spar

In [0]:
#lit() Function to Add Constant Column
data = [("111",50000),("222",60000),("333",40000)]
columns= ["EmpId","Salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()

root
 |-- EmpId: string (nullable = true)
 |-- Salary: long (nullable = true)



In [0]:
from pyspark.sql.functions import when,lit,col
df2 = df.select("*",lit("50").alias("bonus"))
df2.show()

+-----+------+-----+
|EmpId|Salary|bonus|
+-----+------+-----+
|  111| 50000|   50|
|  222| 60000|   50|
|  333| 40000|   50|
+-----+------+-----+



In [0]:
df3 = df.withColumn("dptID",when((col("Salary") >=40000) & (col("Salary") <=50000),lit(500)).otherwise(lit(600)))
df3.show()

+-----+------+-----+
|EmpId|Salary|dptID|
+-----+------+-----+
|  111| 50000|  500|
|  222| 60000|  600|
|  333| 40000|  500|
+-----+------+-----+



In [0]:
#Difference between lit() and typedLit() is that the typedLit() function can handle collection types e.g.: Array, Dictionary(map), etc. Below is an #example usage of typedLit()
from pyspark.sql.functions import typedLit
df4 = df.withColumn("lit_value3", typedLit("flag", StringType()))
df4.show(truncate=False)

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-3189017243643610>:2
      1 #Difference between lit() and typedLit() is that the typedLit() function can handle collection types e.g.: Array, Dictionary(map), etc. Below is an #example usage of typedLit()
----> 2 df4 = df.withColumn("lit_value3", typedLit("flag", StringType()))
      3 df4.show(truncate=False)

ImportError: cannot import name 'typedLit' from 'pyspark.sql.functions' (/databricks/spark/python/pyspark/sql/functions.py)

In [0]:
#In PySpark, the expr() function is used to express complex transformations or computations on columns of a DataFrame using SQL-like expressions. It #allows you to write expressions in a concise and expressive manner. 
from pyspark.sql.functions import expr
data = [("John", 25), ("Alice", 30), ("Bob", 22)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data,columns)
df.show()

+-----+---+
| Name|Age|
+-----+---+
| John| 25|
|Alice| 30|
|  Bob| 22|
+-----+---+



In [0]:
df.withColumn("mail",expr("Name||'_'||Age||'@gmail.com'")).show()

+-----+---+------------------+
| Name|Age|              mail|
+-----+---+------------------+
| John| 25| John_25@gmail.com|
|Alice| 30|Alice_30@gmail.com|
|  Bob| 22|  Bob_22@gmail.com|
+-----+---+------------------+



In [0]:
data = [("James","M"),("Michael","F"),("Jen","")]
columns = ["name","gender"]
df = spark.createDataFrame(data,columns)
df.show()


+-------+------+
|   name|gender|
+-------+------+
|  James|     M|
|Michael|     F|
|    Jen|      |
+-------+------+



In [0]:
df.withColumn("gender",expr("case when gender = 'M' Then 'Male'" + "WHEN gender = 'F' THEN 'Female' ELSE 'Unkown' END")).show()

+-------+------+
|   name|gender|
+-------+------+
|  James|  Male|
|Michael|Female|
|    Jen|Unkown|
+-------+------+



In [0]:
data=[(100,2),(200,3000),(500,500)] 
df=spark.createDataFrame(data).toDF("col1","col2") 
df.filter(expr("col1 == col2")).show()


+----+----+
|col1|col2|
+----+----+
| 500| 500|
+----+----+

